In [3]:
from astroquery.skyview import SkyView
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from astropy.wcs import WCS
import matplotlib
from ipywidgets import widgets
from matplotlib import gridspec
%matplotlib inline
matplotlib.rcParams['font.size']=15

In [2]:
surveys={
    'Optical':['DSS2 IR', 'DSS2 Red','DSS2 Blue'],
    'IOU' :  ['WISE 22','DSS2 Red','GALEX Near UV'],
    'ROR' : ['TGSS ADR1', 'DSS2 Red','NVSS']
}

In [9]:
def make_RGB(position,radius=1,survey='Optical'):
  try:
    if survey=='ROR':
      paths = SkyView.get_images(position=position,pixels=600,scaling="Sqrt",sampler='Lanczos3',radius=radius*u.degree,survey=surveys[survey])
    else:
      paths = SkyView.get_images(position=position,pixels=600,scaling="Log",radius=radius*u.degree,survey=surveys[survey])
    RGB=[]
    for i,path in enumerate(paths):
      img=path[0].data.copy()

      min=img.min()
      img=img-min
      
      max=img.max()
      RGB.append(img/max)
    RGB=np.array(RGB)
    RGB=np.stack((RGB[0],RGB[1],RGB[2]),axis=2)

    wcs = WCS(paths[0][0].header)
  except :
    print(f"{survey} image not found in Server for {position} ")
    RGB=wcs=None
  return RGB,wcs

In [10]:
def init_contour(position,survey,radius=1):
  try:
    if survey=='TGSS':
      paths = SkyView.get_images(position=position,pixels=600,scaling="Sqrt",sampler='Lanczos3',radius=radius*u.degree,survey=['TGSS ADR1'])
      img=paths[0][0].data.copy()
    elif survey=='NVSS':
      paths = SkyView.get_images(position=position,pixels=600,scaling="Sqrt",sampler='Lanczos3',radius=radius*u.degree,survey=['NVSS'])
      img=paths[0][0].data.copy()

    wcs = WCS(paths[0][0].header)
  except:
    print(f"{survey} image not found for {position}")
    img=wcs=None
  return img,wcs

In [6]:
def RAD_RGB(position,radius,name,levels=4):
    # RGB-TGSS
    RGB,wcs=make_RGB(position=position,radius=radius,survey='Optical')
    fig = plt.figure(constrained_layout=True,figsize=(15,15))
    gs  = gridspec.GridSpec(ncols=2, nrows=1, figure=fig)

    if wcs!=None:
        ax=fig.add_subplot(gs[0],projection=wcs)
        ax.imshow(RGB)
        ax.set_autoscale_on(False)

        img,wcs_c= init_contour(position=position, survey='TGSS',radius=radius)
        levels_c=np.arange(0.015,img.max(),(img.max()-0.015)/levels)
        ax.contour(img, transform=ax.get_transform(wcs), colors='white',levels=levels_c)

        ax.set_title(f'Optical RGB with TGSS contours of {position}',color="white")
        ax.annotate("#RGBviaNASAnRADatHomeIndia",(10,10),color='red')
        ax.annotate(f"By {name}",(450,10),color='red')
        ax.axis("off")
        wcs=None

    # IOU-TGSS
    RGB,wcs=make_RGB(position=position,radius=radius,survey='IOU')
    if wcs!=None:
        ax=fig.add_subplot(gs[1],projection=wcs)
        ax.imshow(RGB)

        ax.set_title(f'IOU with TGSS Contours of {position}',color='White')
        ax.annotate("#RGBviaNASAnRADatHomeIndia",(20,20),color='red')
        ax.annotate(f"By {name}",(450,10),color='red')
        ax.set_autoscale_on(False)

        img,wcs_c= init_contour(position=position, survey='TGSS',radius=radius)
        levels_c=np.arange(0.015,img.max(),(img.max()-0.015)/levels)
        ax.contour(img, transform=ax.get_transform(wcs), colors='white',levels=levels_c)
        ax.axis("off")
        plt.show()
        wcs=None

    # ROR-TGSS
    fig = plt.figure(constrained_layout=True,figsize=(15,15))
    gs  = gridspec.GridSpec(ncols=2, nrows=1, figure=fig)
    
    RGB,wcs=make_RGB(position=position,radius=radius,survey='ROR')
    if wcs!=None:
        ax=fig.add_subplot(gs[0],projection=wcs)
        ax.imshow(RGB)

        ax.set_title(f'ROR with TGSS contours of {position}',color="white")
        ax.annotate("#RGBviaNASAnRADatHomeIndia",(10,10),color='red')
        ax.annotate(f"By {name}",(450,10),color='red')
        ax.set_autoscale_on(False)

        img,wcs_c= init_contour(position=position, survey='TGSS',radius=radius)
        levels_c=np.arange(0.015,img.max(),(img.max()-0.015)/levels)
        ax.contour(img, transform=ax.get_transform(wcs), colors='white',levels=levels_c)
        ax.axis("off")
        wcs=None

    # ROR-NVSS
    RGB,wcs=make_RGB(position=position,radius=radius,survey='ROR')
    if wcs!=None:
        ax=fig.add_subplot(gs[1],projection=wcs)
        ax.imshow(RGB)

        ax.set_title(f'ROR with NVSS Contours of {position}',color="white")
        ax.annotate("#RGBviaNASAnRADatHomeIndia",(10,10),color='red')
        ax.annotate(f"By {name}",(450,10),color='red')
        ax.set_autoscale_on(False)

        img,wcs_c= init_contour(position=position, survey='NVSS',radius=radius)
        levels_c=np.arange(0.0015,img.max(),(img.max()-0.0015)/levels)
        ax.contour(img, transform=ax.get_transform(wcs), colors='white',levels=levels_c)
        ax.axis("off")
        plt.show()

In [4]:
file = open("flogo.png", "rb")
image = file.read()
widgets.Image(
    value=image,
    format='png',
    width=300,
    height=400,
)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x12\x00\x00\x00\x9f\x08\x06\x00\x00\x00\xac\x8c0…


# **RGB Maker**

In [7]:
l1= widgets.Label("Your name ")
display(l1)
name = widgets.Text()
display(name)

l2= widgets.Label("Source name or coordinates")
display(l2)
pos =widgets.Text()
display(pos)

l3= widgets.Label("Radius in degrees ")
display(l3)
r = widgets.Text()
display(r)

btn=widgets.Button(description="Submit")
display(btn)

output = widgets.Output()

@output.capture(clear_output=True,wait=True)

def submit(b):
    l4= widgets.Label("Loading images...",font_size=75)
    display(l4)
    try:
        if len(name.value)> 2 and len(pos.value)>2 and len(r.value)>0:
            RAD_RGB(position= pos.value,name=name.value,radius=float(r.value))
        elif len(r.value)> 0 and len(pos.value) >2:
            l5= widgets.Label("Default name set")
            display(l5)
            RAD_RGB(position= pos.value,name="Anonymous",radius=float(r.value)) 
        else:
            l5= widgets.Label("Enter Position and Radius")
            display(l5)
    except:
        l5= widgets.Label("Data not Found")
        display(l5)       
btn.on_click(submit)
output

Label(value='Your name ')

Text(value='')

Label(value='Source name or coordinates')

Text(value='')

Label(value='Radius in degrees ')

Text(value='')

Button(description='Submit', style=ButtonStyle())

Output()